# Install

In [4]:
!apt-get update
# Pick the version that matches /usr/local/cuda (e.g., 12-4 or 12-3)
!apt-get install -y cuda-toolkit-12-8
# or the narrow package:
!apt-get install -y libcurand-dev-12-8

Get:1 http://deb.debian.org/debian bookworm InRelease [151 kB]
Get:2 http://deb.debian.org/debian bookworm-updates InRelease [55.4 kB]
Get:3 http://deb.debian.org/debian-security bookworm-security InRelease [48.0 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/debian12/x86_64  InRelease [1581 B]
Get:5 http://deb.debian.org/debian bookworm/main amd64 Packages [8791 kB]
Get:6 http://deb.debian.org/debian-security bookworm-security/main amd64 Packages [280 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/debian12/x86_64  Packages [938 kB]
Fetched 10.3 MB in 1s (8603 kB/s)

N: Repository 'http://deb.debian.org/debian bookworm InRelease' changed its 'Version' value from '12.11' to '12.12'



The following additional packages will be installed:
  adwaita-icon-theme at-spi2-common at-spi2-core ca-certificates-java cuda-command-line-tools-12-8
  cuda-compiler-12-8 cuda-cuobjdump-12-8 cuda-cupti-12-8 cuda-cupti-dev-12-8 cuda-cuxxfilt-12-8
  cuda-documentatio

In [5]:
!ls -l /usr/local/cuda/include/curand.h
!nvcc --version

-rw-r--r-- 1 root root 44075 Feb 13  2025 /usr/local/cuda/include/curand.h
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [6]:
%uv pip install pandas pyarrow llama-cpp-python faiss-cpu sentence-transformers pandas pyarrow openpyxl tqdm numpy rapidfuzz
# (Optional GPU for embeddings only)
%uv pip install pandas pyarrow vllm bitsandbytes
%uv pip install -U "vllm[flashinfer]"
%uv pip install torch --index-url https://download.pytorch.org/whl/cu121

Using Python 3.12.6 environment at: /usr/local
Resolved 58 packages in 1.50s
Building llama-cpp-python==0.3.16
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
openpyxl   ------------------------------ 14.88 KiB/245.03 KiB
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
openpyxl   ------------------------------ 14.88 KiB/245.03 KiB
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
et-xmlfile ------------------------------     0 B/17.64 KiB
openpyxl   ------------------------------ 14.88 KiB/245.03 KiB
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
et-xmlfile ------------------------------ 14.88 KiB/17.64 KiB
openpyxl   ------------------------------ 14.88 KiB/245.03 KiB
Building llama-cpp-python==0.3.16
⠙ Preparing packages... (0/8)
et-xmlfile ------------------------------ 14.88 KiB/17.64 KiB
diskc

# RAG

In [34]:
!nvidia-smi

Mon Sep 29 18:10:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:CB:00.0 Off |                    0 |
| N/A   36C    P0            120W /  700W |    1915MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!python clean.py build \
  --snomed_parquet snomed_all_data.parquet \
  --rxnorm_parquet rxnorm_all_data.parquet \
  --out_dir indices \
# --hf_token .... # (if using private models from HuggingFace)

/usr/local/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[HF] Logged in to HuggingFace Hub.
[Build] Embedding model: google/embeddinggemma-300m (dim=768, device=cuda)
[Build] SNOMED rows: 1,035,233
[Build] SNOMED add: 100%|███████████████████| 1011/1011 [06:10<00:00,  2.73it/s]
[Build] Saved SNOMED index.
[CUDA] Cleared after SNOMED phase | allocated=1.18 GB, reserved=1.19 GB
[CUDA] Cleared after model->CPU | allocated=0.03 GB, reserved=0.06 GB
[CUDA] Cleared after model->CUDA | allocated=1.18 GB, reserved=1.19 GB
[Build] RxNorm rows: 379,991
[Build] RxNorm add: 100%|█████████████████████| 372/372 [05:23<00:00,  1.15it/s]
[Build] Saved RxNorm index.
[Build] Done. Artifacts in: indices


In [33]:
!python clean.py batch \
  --index_dir indices \
  --in_file Test.xlsx --out_file out.xlsx \
  --use_llm_clean false --use_llm_rerank false \
  --llm_backend auto \
  --include_topk

/usr/local/lib/python3.12/site-packages/torch/cuda/__init__.py:63: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml  # type: ignore[import]
[Batch] Legacy ANN: 100%|█████████████████████| 400/400 [00:14<00:00, 26.81it/s]
[Batch] Wrote predictions to: out.xlsx
